In [1]:
import cv2
import scipy
import numpy as np
import trimesh
from scipy.spatial import ConvexHull


source = 'hat'

img = cv2.imread('./images/'+source+'.png')
out = cv2.imread('./images/'+source+'.png')

print('img.shape:',img.shape)

height = img.shape[0]
width = img.shape[1]
c = img.shape[2]


flattened_img = img.reshape(height*width,3)
#print(flattened_img.shape) # (51400, 3)


hull = ConvexHull(flattened_img)
                  #, shape = (51400,2))
    
#print(hull.points)

#print(len(hull.simplices))

palette = np.zeros((len(hull.simplices)*3,3))
# (128*3) points: [128*3][3]

#print(palette.shape)

# 0~128
for index in range (len(hull.simplices)):
    #print(hull.simplices[index])
    #print(flattened_img[(hull.simplices[index][0])])
    palette[index] = flattened_img[(hull.simplices[index][0])]
    #print(flattened_img[(hull.simplices[index][1])])
    palette[len(hull.simplices)*1+index] = flattened_img[(hull.simplices[index][1])]
    #print(flattened_img[(hull.simplices[index][2])])
    palette[len(hull.simplices)*2+index] = flattened_img[(hull.simplices[index][2])]
    
#print(palette)   
#palette = np.unique(palette, axis=0)

print('palette size:',palette.shape)



'''
# print palette colors to 'out', each color occupy n lines, n = 'lines'
lines = height//palette.shape[0]
for ct in range(palette.shape[0]):
    for x in range(width):
        for i in range(lines):
            out[ct*lines+i][x] = palette[ct]

cv2.imwrite('palette1.png',out)
show(out)

'''    






img.shape: (778, 650, 3)
palette size: (336, 3)


"\n# print palette colors to 'out', each color occupy n lines, n = 'lines'\nlines = height//palette.shape[0]\nfor ct in range(palette.shape[0]):\n    for x in range(width):\n        for i in range(lines):\n            out[ct*lines+i][x] = palette[ct]\n\ncv2.imwrite('palette1.png',out)\nshow(out)\n\n"

In [2]:
# stroke density

import trimesh

# find barycentre of palette, g
#print(palette)

g = np.mean(flattened_img, axis=0)#sum(palette) // len(palette)


# Estimate the stroke density map.
intersector = trimesh.Trimesh(faces=hull.simplices, vertices=hull.points).ray
start = np.tile(g[None, :], [height * width, 1])  #construct an array with all elements be the 'center'
direction = flattened_img - start  # compute each direction

print('Begin ray intersecting ...')  
index_tri, index_ray, locations = intersector.intersects_id(start, direction, return_locations=True, multiple_hits=True)
    # Find the intersections between the current mesh and an array of rays. 
    # return 1. Index of triangles hit
    #        2. Index of ray that hit triangle
    #        3. Position of intersection in space 
print('Intersecting finished.')

hits = np.zeros(shape=(height * width, c), dtype=np.float32)
hits_count = np.zeros(shape=(height * width, 1), dtype=np.float32)

CI = index_ray.shape[0]
    
for c in range(CI):
    i = index_ray[c]
    hits_count[i] += 1
    hits[i] += locations[c]
    
hits = (hits + 1e-10) / (hits_count + 1e-10)
hits = hits.reshape((height, width, 3))
hits_count = hits_count.reshape((height, width))

hits[hits_count < 1] = img[hits_count < 1]

# |g - hp|
intersection_distance = np.sqrt(np.sum(np.square(hits - g), axis=2, keepdims=True))

# |cp - hp|
pixel_distance = np.sqrt(np.sum(np.square(img - g), axis=2, keepdims=True))

stroke_density = np.abs(pixel_distance) / np.abs(intersection_distance)   #(1.0 - np.abs(1.0 - pixel_distance / intersection_distance))
stroke_density = (stroke_density * 1.2).clip(0, 1)
stroke_density *= 255

cv2.imwrite('stroke_density_'+source+'.png', stroke_density.clip(0, 255).astype(np.uint8))







Begin ray intersecting ...
Intersecting finished.


True

In [1]:
def show(img):
    cv2.imshow('My Image',img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)